https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524

In [ ]:
!pip install textblob
!pip install findspark

In [ ]:
# import necessary packages
import os
import json
import time
import subprocess
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob

from IPython.display import clear_output

In [ ]:
findspark.init()

In [ ]:
# # Create streaming server process
# server_process = subprocess.Popen(["python", r"tweet_stream_producer.py", "Test"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# # Check is listening
# with server_process.stdout:
#     for line in iter(server_process.stdout.readline, b''): 
#         if "Listening" in str(line):
#             print(line)
#             break
#         else:
#             server_process.terminate()
#             print("Process terminated, Return code:", server_process.returncode)
#             print("Something went wrong!")

In [ ]:
# # Kill server process
# server_process.terminate()
# print("Process terminated, Return code:", server_process.returncode)
# server_process = None

In [ ]:
def json_load(text):
    return json.loads(text)

In [ ]:
def get_message(text):
    msg = json.loads(text)
    # if tweet is longer than 140 characters
    if "extended_tweet" in msg:
        return str(msg['extended_tweet']['full_text'])
    else:
        return str(msg['text'])

In [ ]:
def get_tweet_field(text, field):
    msg = json.loads(text)
    if '/' in field:
        fields = field.split('/')
        fieldDepth = len(fields)
        f = msg
        for i in range(fieldDepth):
            if(f is None):
                return None
            if i == fieldDepth - 1:
                return str(f[fields[i]]) 
            else:
                f = f[fields[i]]
    else:
        return str(msg[field])

In [ ]:
def get_analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
def preprocessing(tweets):
    
#     # Convert to json
#     json_load_udf = udf(json_load, StringType())
#     tweets = tweets.withColumn("json", json_load_udf("value"))
    
    # Get id
    get_tweet_field_udf = udf(get_tweet_field, StringType())
    tweets = tweets.withColumn("id", get_tweet_field_udf("value", lit('id')))
    
    # Get created_at
    get_tweet_field_udf = udf(get_tweet_field, StringType())
    tweets = tweets.withColumn("created_at", get_tweet_field_udf("value", lit('created_at')))
    
    # Get place full name
    get_tweet_field_udf = udf(get_tweet_field, StringType())
    tweets = tweets.withColumn("place_full_name", get_tweet_field_udf("value", lit('place/full_name')))
    
    # Get place country
    get_tweet_field_udf = udf(get_tweet_field, StringType())
    tweets = tweets.withColumn("place_country", get_tweet_field_udf("value", lit('place/country')))
    
    # Get place country code
    get_tweet_field_udf = udf(get_tweet_field, StringType())
    tweets = tweets.withColumn("place_country_code", get_tweet_field_udf("value", lit('place/country_code')))
    
    # Get message
    get_message_udf = udf(get_message, StringType())
    tweets = tweets.withColumn("message", get_message_udf("value"))
    
    # Get cleaned words from message for analysis
    #tweets = tweets.na.replace('', None)
    #tweets = tweets.na.drop()
    tweets = tweets.withColumn('words', tweets.message)
    tweets = tweets.withColumn('words', F.regexp_replace('words', r'http\S+', ''))
    tweets = tweets.withColumn('words', F.regexp_replace('words', '@\w+', ''))
    tweets = tweets.withColumn('words', F.regexp_replace('words', '#', ''))
    tweets = tweets.withColumn('words', F.regexp_replace('words', 'RT', ''))
    tweets = tweets.withColumn('words', F.regexp_replace('words', ':', ''))
    
    # Drop unnesscessary data
    tweets = tweets.drop("value")
    #tweets = tweets.drop("json")
    
    return tweets

In [ ]:
# Text classification using TextBlob
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity
def text_classification(tweets):
    
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    tweets = tweets.withColumn("polarity", polarity_detection_udf("words"))
    
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, StringType())
    tweets = tweets.withColumn("subjectivity", subjectivity_detection_udf("words"))
    
    # analysis
    get_analysis_udf = udf(get_analysis, StringType())
    tweets = tweets.withColumn('analysis', get_analysis_udf('polarity'))
    
    return tweets

In [ ]:
#import libraries to visualize the results from stream
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas

def visualize_results_from_stream():

    get_ipython().run_line_magic('matplotlib', 'inline')
    count = 0
    while count < 5:

        time.sleep(5)
        top_10_tags = sqlContext.sql( 'Select hashtag, count from tweets' )
        top_10_df = top_10_tags.toPandas()
        display.clear_output(wait=True)
        plt.figure( figsize = ( 10, 8 ) )
        sns.barplot( x="count", y="hashtag", data=top_10_df)
        plt.show()
        count = count + 1
        #     print(count)

In [ ]:
# import necessary packages
import socket
import pandas as pd
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc, explode, split

spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

#sc = SparkContext()
sc = spark.sparkContext
ssc = StreamingContext(sc, 5)
#socket_stream = ssc.socketTextStream(socket.gethostname(), 5555)
# raw_stream = spark.readStream \
#                     .format("socket") \
#                     .option("host", socket.gethostname()) \
#                     .option("port", 5555) \
#                     .load()
raw_stream = ssc.readStream.text('gs://cloud-project-bucket-22/Data/*').load()
#sqlContext = SQLContext(sc)

# lines of tweets with socket_stream window of size 10, or 10 #seconds windows of time
lines = raw_stream.window(10)
#lines = raw_stream


In [ ]:
def process_row(row):
    print(row)
    
def process_batch(df, epoch_id):
    clear_output(wait=True)
    print('Process batch...')
    print(df.head(5))

In [ ]:
 # Start running the query that prints the running counts to the console
tweets = lines
tweets = preprocessing(lines)
tweets = text_classification(tweets)

# Extract analytics here

# Visualize the query
query = tweets.writeStream.foreachBatch(process_batch).start()
query.awaitTermination()

# Mutiple streams
# val query1 = df.writeStream.start()
# val query2 = df.writeStream.start()

# spark.streams.awaitAnyTermination()

In [ ]:
# just a tuple to assign names
#from collections import namedtuple
#fields = ("hashtag", "count" )
#Tweet = namedtuple( 'Tweet', fields )
#tweets = lines.map(lambda tweet: (json.loads(tweet)['id'], json.loads(tweet)['text']))
#tweets = lines.map(lambda tweet: (json.loads(tweet)['id'], json.loads(tweet)['text']))
#tweets.foreachRDD( lambda rdd: rdd.toDF()):
#lines.foreachRDD(process_batch)
#process_batch(lines).start()
#words = lines.flatMap(lambda text: text.split(" "))
#pairs = words.map(lambda word: ( word.lower(), 1 ))
#wordCounts = pairs.reduceByKey( lambda a, b: a + b ) 
#wordCounts.pprint()
#tweets.print()


#.registerTempTable("tweets")))

# # here we apply different operations on the tweets and save them to #a temporary sql table
# (lines.map(lambda tweet: Tweet(json.loads(text)['id'], json.loads(text)['text']))
# .flatMap( lambda text: text.split( " " ))
#     # Lower cases the word
#     .map( lambda word: ( word.lower(), 1 ) )
#     .reduceByKey( lambda a, b: a + b )
#     # Stores in a Tweet Object
#     .map( lambda rec: Tweet( rec[0], rec[1] ) )
#     # Sorts Them in a dataframe
#     .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
#     .limit(10).registerTempTable("tweets") ) )

    # Registers only top 10 words to a table.
#     ( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
#       # Checks for    hashtag calls  
#       #.filter( lambda word: word.lower().startswith("#") ) 
#       .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
#       .reduceByKey( lambda a, b: a + b ) 
#      # Stores in a Tweet Object
#       .map( lambda rec: Tweet( rec[0], rec[1] ) )
#      # Sorts Them in a dataframe
#       .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
#      # Registers only top 10 hashtags to a table.
#       .limit(10).registerTempTable("tweets") ) )



In [ ]:
# start streaming and wait couple of minutes to get enought tweets
#ssc.start()
#ssc.awaitTermination()

In [ ]:
# stop streaming and wait couple of minutes to get enought tweets
#ssc.stop(stopSparkContext=True, stopGraceFully=False)